In [1]:
import polars as pl
from polimi.utils._catboost import reduce_polars_df_memory_size
from polars import testing
import gc

RANKER = True
TRAIN_VAL = True
dataset_path = '/home/ubuntu/experiments/preprocessing_train_new'
validation_path = '/home/ubuntu/experiments/preprocessing_validation_new'
batch_split_directory = '/home/ubuntu/experiments/batches_train_val_new/batches'

model_path = '/home/ubuntu/experiments/batches_train_val_new/models'


EVAL = False
SAVE_PREDICTIONS = False
N_BATCH = 10
BATCHES = [1, 2, 3, 4, 5, 6, 7, 8, 9]


def load_batch(dataset_path, batch_split_directory, batch_index):

    train_ds = pl.scan_parquet(dataset_path + '/train_ds.parquet')

    if TRAIN_VAL:
        val_ds = pl.scan_parquet(validation_path + '/validation_ds.parquet')
    batch = pl.scan_parquet(batch_split_directory +
                            f'/batch_{batch_index}.parquet').collect()

    subsampled_train = train_ds.filter(pl.col('impression_id').is_in(
        batch.select('impression_id'))).collect()
    columns = subsampled_train.columns

    if TRAIN_VAL:
        subsampled_val = val_ds.filter(pl.col('impression_id').is_in(
            batch.select('impression_id'))).select(columns).collect()
        subsampled_train = pl.concat(
            [subsampled_train, subsampled_val], how='vertical_relaxed')

    if 'postcode' in subsampled_train.columns:
        subsampled_train = subsampled_train.with_columns(
            pl.col('postcode').fill_null(5))
    if 'article_type' in subsampled_train.columns:
        subsampled_train = subsampled_train.with_columns(
            pl.col('article_type').fill_null('article_default'))

    subsampled_train = subsampled_train.sort(by='impression_id')
    groups = subsampled_train.select('impression_id').to_numpy().flatten()
    subsampled_train = subsampled_train.drop(
        ['impression_id', 'article', 'user_id', 'impression_time']).to_pandas()

    X = subsampled_train.drop(columns=['target'])
    y = subsampled_train['target']
    print(X.shape)

    if 'impression_time' in X:
        X = X.drop(['impression_time'])

    del train_ds, batch, subsampled_train
    gc.collect()

    return X, y, groups

2024-05-31 08:25:34.877713: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-31 08:25:34.921010: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 08:25:35.590083: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
X, y, groups = load_batch(dataset_path, batch_split_directory, 1)

(28420756, 296)


In [3]:
X

,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,trendiness_score_3d,weekday,hour,...,mean_JS_l_inf_user,std_JS_l_inf_user,mean_topic_model_cosine_l_inf_user,topics_cosine_l_inf_user,article_delay_hours_l_inf_article,mean_JS_l_inf_article,std_JS_l_inf_article,mean_topic_model_cosine_l_inf_article,topics_cosine_l_inf_article,endorsement_20h_articleuser_l_inf_articleuser
0,2,16.0,NaN,False,2,NaN,False,395,7,21,...,0.499210,0.453403,0.920897,0.647336,0.012048,0.245138,0.407771,0.882152,0.644699,1.0
1,2,16.0,NaN,False,2,NaN,False,72,7,21,...,0.088987,0.354481,0.882601,0.956521,0.024096,0.041950,0.195826,0.831786,0.952625,1.0
2,2,16.0,NaN,False,2,NaN,False,295,7,21,...,0.404086,0.386142,0.869966,0.777281,0.118280,0.245222,0.283392,0.777757,0.774115,1.0
3,2,16.0,NaN,False,2,NaN,False,282,7,21,...,0.390346,0.481760,0.913757,0.858693,0.023810,0.133343,0.306269,0.866335,0.855195,1.0
4,2,16.0,NaN,False,2,NaN,False,554,7,21,...,0.774656,0.964732,0.949907,0.320483,0.012048,0.218484,0.532947,0.876442,0.319178,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28420751,1,26.0,NaN,False,2,NaN,False,512,4,10,...,0.325236,0.514818,0.816969,0.853378,0.000000,0.081166,0.357610,0.745813,0.835396,1.0
28420752,1,26.0,NaN,False,2,NaN,False,243,4,10,...,0.609505,0.671289,0.789678,0.618911,0.023810,0.149527,0.339907,0.721788,0.605869,1.0
28420753,1,26.0,NaN,False,2,NaN,False,666,4,10,...,0.434137,0.545898,0.913322,0.741515,0.006061,0.116484,0.414622,0.830918,0.725890,1.0
28420754,1,26.0,NaN,False,2,NaN,False,344,4,10,...,0.471949,0.563019,0.876786,0.830871,0.985745,0.167889,0.467627,0.787589,0.813363,0.2
